In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "5"
import os.path as path
from copy import copy

import numpy as np
from joblib import Parallel, delayed

from ExperimentConfig import ExperimentConfig
from Experiment import Experiment
from CombinedExperiment import CombinedExperiment
from typing import List
from Experiment import RaceResult

class ComparisonResult:
    def __init__(self, results: List[RaceResult]):
        self.results = results

base_config = ExperimentConfig("base_config",
                         "IRV",
                         equal_pct_bins=True,
                         candidate_variance=.5,
                         quality_variance=0,
                         ideology_flexibility = .7,
                         sampling_voters=1000,
                         model_path=f"exp/v26/IRV-A.mdl.010000.progress")

def run_strategic_races(irv_config: ExperimentConfig, h2h_config: ExperimentConfig):
    candidates = base_config.gen_candidates(5)
    voters = irv_config.population.generate_unit_voters(irv_config.sampling_voters)
    def run_config(c: ExperimentConfig) -> RaceResult:
        x = Experiment(c)
        return x.run_strategic_race_c(candidates, voters)

    return [run_config(c) for c in [irv_config, h2h_config]]

def compute_SUE_single(rr: RaceResult):
    return CombinedExperiment.compute_SUE_single_e(rr)

import matplotlib.pyplot as plt
def make_line_plot(data, title, labels,
                   xlabel: str = "Candidate Ideological Flexibility (stddev)",
                   ylabel: str = "Social Utility Efficiency"):
    import matplotlib as mpl
    mpl.rcParams['figure.dpi'] = 300
    n_rows = 1
    n_cols = 1
    fig, axis = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(20, 10))
    fig.suptitle(title, color="black", fontsize=22)
    fig.set_facecolor("white")

    count = 0
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    axis.tick_params(axis='x', colors="black")

    for i in range(len(data)):
        d = data[i]
        plt.plot(d[0], d[1], label =labels[i])
    plt.legend()

    axis.set_xlabel(xlabel, fontsize=20)
    axis.set_ylabel(ylabel, fontsize=20)

import pickle
def save_results(file_path: str, obj):
    with open(file_path, "wb") as f:
        pickle.dump(obj, f)

def load_results(file_path: str):
    if path.exists(file_path):
        with open(file_path, "rb") as f:
            return pickle.load(f)
    else:
        return None

In [5]:
n_voters = 1000
irv_config = copy(base_config)
irv_config.election_name = "IRV"
irv_config.model_path = "exp/v26/IRV-A.mdl.050000.progress"
irv_config.sampling_voters = 1000

h2h_config = copy(base_config)
h2h_config.election_name="H2H"
h2h_config.model_path = "exp/v26/H2H-A.mdl.050000.progress"
h2h_config.sampling_voters = 1000

n_races = 1
# results: List[List[RaceResult]] = Parallel(n_jobs=32)(
#     delayed(run_strategic_races)(irv_config, h2h_config) for _ in range(n_races))
results = [run_strategic_races(irv_config, h2h_config) for _ in range(n_races)]


irv_count = 0
h2h_count = 0
equal_count = 0

for rr in results:
    irv_sue = compute_SUE_single(rr[0])
    h2h_sue = compute_SUE_single(rr[1])
    print("irv base candidates:  ", end = '')
    for c in rr[0].base_candidates:
        print("% 6.2f " % c.ideology.vec[0], end = "")
    print("")

    print("h2h base candidates:  ", end = '')
    for c in rr[1].base_candidates:
        print("% 6.2f " % c.ideology.vec[0], end = "")
    print("")

    print("irv_sue % 6.2f % 6.2f h2h_sue % 6.2f % 6.2f" % (irv_sue, rr[0].winner.ideology.vec[0], h2h_sue, rr[1].winner.ideology.vec[0]))
    if irv_sue > h2h_sue:
        irv_count += 1
    elif h2h_sue > irv_sue:
        h2h_count += 1
    else:
        equal_count += 1

print("irv_count %d h2h_count %d equal_count %d" % (irv_count, h2h_count, equal_count))

loading model: exp/v26/IRV-A.mdl.050000.progress
starting candidates IRV
	candidate c-0   0.34
	candidate c-1   0.51
	candidate c-2   0.57
	candidate c-3  -0.50
	candidate c-4  -0.26
c-0: bin 10 bin_start: -0.05 ideology   0.04  win_probability 0.037 win_bonus 0.563 return 0.021
c-0: bin 11 bin_start:  0.05 ideology   0.12  win_probability 0.054 win_bonus 0.677 return 0.037
c-0: bin 12 bin_start:  0.16 ideology   0.26  win_probability 0.035 win_bonus 0.878 return 0.031
c-0: bin 13 bin_start:  0.26 ideology   0.33  win_probability 0.093 win_bonus 0.971 return 0.090
c-0: bin 14 bin_start:  0.37 ideology   0.41  win_probability 0.119 win_bonus 0.901 return 0.107 best!
c-0: bin 15 bin_start:  0.48 ideology   0.56  win_probability 0.096 win_bonus 0.700 return 0.067
c-0: bin 16 bin_start:  0.60 ideology   0.72  win_probability 0.028 win_bonus 0.467 return 0.013
c-1: bin 12 bin_start:  0.16 ideology   0.24  win_probability 0.104 win_bonus 0.614 return 0.064
c-1: bin 13 bin_start:  0.26 ideolo

In [6]:
wid = -0.12
wu = CombinedExperiment.estimate_utility(wid)
id = [-0.12, -0.88,  0.76, -0.16, -0.48 ]
uu = [CombinedExperiment.estimate_utility(i)  for i in id]
au = np.average(uu)
mu = np.max(uu)
sue = (wu - au) / (mu - au)
sue

1.0199388124239739